In [ ]:
# Enhanced configuration for RAG augmented experiment
MODEL_KEY = 'qwen25'
TOP_K_RULES = 3
MAX_NEW_TOKENS = 128
EXTRA_PROMPT = ''
OUTPUT_DIR = 'outputs'
from utils import ensure_dir
ensure_dir(OUTPUT_DIR)


# Experiment 3: RAG-Augmented Action Decisions

Inject top-k retrieved driving rules (CLIP similarity) before asking for action decision. Compare outputs vs non-RAG.

In [ ]:
MODEL_KEY = 'qwen25'
TOP_K_RULES = 3
MAX_NEW_TOKENS = 128
EXTRA_PROMPT = ''

In [ ]:
from PIL import Image
from utils import list_frame_pairs, build_single_view_prompt, build_rag_prompt, generate_action, retrieve_rules_for_image, parse_action_json
pairs = list_frame_pairs()
pairs

In [ ]:
baseline_prompt = build_single_view_prompt(EXTRA_PROMPT)
rag_outputs = []
baseline_outputs = []
parsed_rag = []
parsed_base = []
for cam_path, map_path in pairs:
    cam = Image.open(cam_path).convert('RGB')
    ranked = retrieve_rules_for_image(cam, top_k=TOP_K_RULES)
    rag_prompt = build_rag_prompt(baseline_prompt, ranked, max_rules=TOP_K_RULES)
    base_out = generate_action(MODEL_KEY, [cam], baseline_prompt)
    rag_out = generate_action(MODEL_KEY, [cam], rag_prompt)
    baseline_outputs.append(base_out)
    rag_outputs.append(rag_out)
    parsed_base.append(parse_action_json(base_out) or {})
    parsed_rag.append(parse_action_json(rag_out) or {})
len(rag_outputs), len(baseline_outputs)

In [ ]:
parsed_rag

In [ ]:
# Simple comparison: count action changes introduced by rules
baseline_actions = [p.get('action','?') for p in parsed_base]
rag_actions = [p.get('action','?') for p in parsed_rag]
changes = sum(1 for b,r in zip(baseline_actions, rag_actions) if b != r)
print({'action_changes_due_to_rag': changes})

In [ ]:
# Persist baseline vs RAG comparison summary
from utils import save_text
if 'parsed_rag' in globals():
    import json, os
    base_actions = [p.get('action','?') for p in parsed_base]
    rag_actions = [p.get('action','?') for p in parsed_rag]
    changes = sum(1 for b,r in zip(base_actions, rag_actions) if b != r)
    summary = {
        'baseline_actions': base_actions,
        'rag_actions': rag_actions,
        'action_changes_due_to_rag': changes
    }
    save_text(os.path.join(OUTPUT_DIR,'experiment3_rag_summary.json'), json.dumps(summary, indent=2))
    summary
else:
    print('Run generation cell first.')